In [ ]:
# IMPORT NECESSARY BUILT IN LIBRARIES
!pip install contractions
!pip install tqdm
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_lg-0.5.0.tar.gz
!pip install -U spacy
!pip install mlxtend
!pip install apyori
!pip install yellowbrick
!pip install texthero -q
!pip install textblob
!python -m spacy download en_core_web_sm
from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures
from nltk.metrics import BigramAssocMeasures
from yellowbrick.cluster import SilhouetteVisualizer
from mlxtend.preprocessing import TransactionEncoder
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_sci_lg
import string
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import contractions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
import string
from mlxtend.frequent_patterns import apriori, association_rules
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
%matplotlib inline

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Shilpi\\anaconda3\\Lib\\site-packages\\~3acy\\attrs.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_lg-0.5.0.tar.gz (532.3 MB)
  Using cached spacy-3.2.4-cp39-cp39-win_amd64.whl (11.3 MB)
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached thinc-8.0.17-cp39-cp39-win_amd64.whl (1.0 MB)
  Using cached srsly-2.4.4-cp39-cp39-win_amd64.whl (450 kB)
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.1
    Uninstalling catalogue-1.0.1:
      Successfully uninstalled catalogue-1.0.1
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.5
    Uninstalling thinc-7.4.5:
      Successfully uninstalled thinc-7.4.5
  Attempting uninstall: spacy
    Found existing installation: spacy 2.3.7
    Uninstalling spacy-2.3.7:
      Successfully uninstalled spacy-2.3.7
  Using cached spacy-3.4.1-cp39-

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 2.3.1 requires spacy<2.4.0,>=2.3.0, but you have spacy 3.4.1 which is incompatible.
en-core-sci-lg 0.5.0 requires spacy<3.3.0,>=3.2.3, but you have spacy 3.4.1 which is incompatible.


# Read the data and add header to column, Since we need ton consider only titles. Lets give header name as 'Title'

In [ ]:
df = pd.read_csv("CORD19-10k.csv", 
                  sep='\t', 
                  names=["Title"])

# Lets see some data from title column

In [ ]:
df.head()

In [ ]:
# From above we can see we have a column in proper format

# Lets see size of data

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# From above we see we have 1000 rows of data

# Data preprocessing

In [ ]:
# Lets find the lenght of the title

In [ ]:
df["length of message"] = df["Title"].apply(len)

In [ ]:
df.head()

# Expanding Contractions

Contactions ate those little literary shortcuts we take where instead of "Should haave" we prefer "Should've" or where "Do not" quickly become Don't. We will add new coloumn to our dataframe called "Title_no_contract" aand apply laambda function to the "Title" filed which will expand any contactions.

In [ ]:
df['title_with_no_contract']=df['Title'].apply(lambda x:[contractions.fix(word) for word in x.split()])
df.head()

Convert title_with_no_contract coloumn into string to proceed further

In [ ]:
df['title_with_no_contract']=df['title_with_no_contract'].astype('string')
df.head()

# Tokenization
tokenization will split each individual word into a token, we will apply NLTK.word_tokenize() function and create aa new columns names ='tokenized'

In [ ]:
df['title_tokenized']=df['title_with_no_contract'].apply(word_tokenize)
df.head()

# Converting all characters to Lowercase

Tarnsforming all words to lowercase is very common pre processing step. We will append a new coloumn names "lower" to the dataframe which will transfom all the tokenized words into lowercase

In [ ]:
df['title_in_lowercase']=df['title_tokenized'].apply(lambda x:[word.lower() for word in x])
df.head()

# Removing punctuations - 

it is often removed from our data since they serve little value to the data. we will create a new coloumn which a the puntuation removed

In [ ]:
punc=string.punctuation
df['title_with_no_punc']=df['title_in_lowercase'].apply(lambda x:[word for word in x if word not in punc])
df.head()

# Removing stop words

We ultimately would want the expanded contractions to be tokenized seperaately. There fore lets convert the lists under the "Title_no_contract" column back into strings

In [ ]:
stop_words=set(stopwords.words('english'))
df['title_with_stopwords_removed']=df['title_with_no_punc'].apply(lambda x:[word for word in x if word not in stop_words])
df.head()

# Finally we get clean data after various preprocessing which we will use further 

In [ ]:
df['Clean_Data']=df['title_with_stopwords_removed'].astype('string')


# Lets drop null values if any

In [ ]:
df.dropna(inplace=True)
df.info()

# From above we can see there is no null vaues

# Convert titles into corresponding feature vectors 

In [ ]:
punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%"]
stop_words = text.ENGLISH_STOP_WORDS.union(punc)
desc = df['Clean_Data'].values
vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(desc)
X

td_arr=X.toarray()
print(len(td_arr))
td_arr

df_in= pd.DataFrame(td_arr)

df_in.info()

df_in.head()

In [ ]:
word_features = vectorizer.get_feature_names()
print(len(word_features))
print(word_features[5000:5100])

In [ ]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')

def tokenize(text):
    return [stemmer.stem(word) for word in tokenizer.tokenize(text.lower())]

In [ ]:
vectorizer2 = TfidfVectorizer(stop_words = stop_words, tokenizer = tokenize)
X2 = vectorizer2.fit_transform(desc)
word_features2 = vectorizer2.get_feature_names()
print(len(word_features2))
print(word_features2[:50]) 

In [ ]:
vectorizer3 = TfidfVectorizer(stop_words = stop_words, tokenizer = tokenize, max_features = 1000)
X3 = vectorizer3.fit_transform(desc)
words = vectorizer3.get_feature_names()
X3[0]

#  From above we got  feature vectors which is a sparse matrix with 64948 stored elements in Compressed Sparse Row format

# Converting title text into corresponding TF-IDF vector

Lets Calculate Term Frequency

In [ ]:
def term_frequency(df):
    tf1 = (df['Title'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
    tf1.columns = ['words','tf']
    return tf1.head()

In [ ]:
term_frequency(df)

In [ ]:
tf1 = (df['Title'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1.head()

# Term Frequency – Inverse Document Frequency (TF-IDF)

In [ ]:
tf1 = (df['Title'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']

In [ ]:
for i,word in enumerate(tf1['words']):
    tf1.loc[i, 'idf'] = np.log(df.shape[0]/(len(df[df['Title'].str.contains(word)])))

In [ ]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

In [ ]:
# From above we can see TF-IDF

# PCA - Reducing sparse matrix into 2D

In [ ]:
# For sparse matrix we can either use SparsePCA or TruncatedSVD for sparse matrix.

In [ ]:
from sklearn.decomposition import SparsePCA
transformer = SparsePCA(n_components=2, random_state=0)
transformer.fit(X3.toarray())
X_transformed = transformer.transform(X3.toarray())
X_transformed
X_transformed.shape

In [ ]:
from sklearn.decomposition import TruncatedSVD 
svd = TruncatedSVD(n_components=2, random_state=42)
data = svd.fit_transform(X3.toarray()) 
data.shape

# Dimensionality Reduction for 2D Plotting

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(verbose=1, perplexity=50)  # Changed perplexity from 100 to 50 per FAQ
X_embedded = tsne.fit_transform(data)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

# sns settings
sns.set(rc={'figure.figsize':(15,15)})

# colors
palette = sns.color_palette("bright", 1)

# plot
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], palette=palette)
plt.title('t-SNE with no Labels')
plt.savefig("t-sne_covid19.png")
plt.show()

# 2. Perform K-means clustering for different values of K (2,3,4,5) and evaluate the quality of clustering using Silhouette score

In [ ]:
# Before doing k mean clustering lets find the optimul number of cluster by using elbow method

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=10,random_state=0)
    kmeans.fit(X3)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.savefig('elbow.png')
plt.show()

# From above it seems 2 number of cluster will be optimal

# K Mean clustering for K=2 and evaaluating quality using Silhouette score

In [ ]:
kmeans = KMeans(n_clusters = 2, init = "k-means++", random_state = 42)
y_kmeans = kmeans.fit_predict(X3)
y_kmeans

In [ ]:
print(f'Silhouette Score(n=2): {silhouette_score(data, y_kmeans)}')

# K Mean clustering for K=3 and evaaluating quality using Silhouette score

In [ ]:
kmeans = KMeans(n_clusters = 3, init = "k-means++", random_state = 42)
y_kmeans = kmeans.fit_predict(X3)
y_kmeans

In [ ]:
print(f'Silhouette Score(n=3): {silhouette_score(X3, y_kmeans)}')

# K Mean clustering for K=4 and evaaluating quality using Silhouette score

In [ ]:
kmeans = KMeans(n_clusters = 4, init = "k-means++", random_state = 42)
y_kmeans = kmeans.fit_predict(X3)
y_kmeans

In [ ]:
print(f'Silhouette Score(n=4): {silhouette_score(X3, y_kmeans)}')

# K Mean clustering for K=5 and evaaluating quality using Silhouette score

In [ ]:
kmeans = KMeans(n_clusters = 4, init = "k-means++", random_state = 42)
y_kmeans = kmeans.fit_predict(X3)
y_kmeans

In [ ]:
print(f'Silhouette Score(n=4): {silhouette_score(X3, y_kmeans)}')

# For each value of K, plot clusters (all cluster points in same cluster with same color, cluster points in different cluster in different colors). [1 pt]

# For K=2

In [ ]:
plt.scatter(data[y_kmeans==0, 0], data[y_kmeans==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(data[y_kmeans==1, 0], data[y_kmeans==1, 1], s=100, c='blue', label ='Cluster 2')

# For K=3

In [ ]:
plt.scatter(data[y_kmeans==0, 0], data[y_kmeans==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(data[y_kmeans==1, 0], data[y_kmeans==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(data[y_kmeans==2, 0], data[y_kmeans==2, 1], s=100, c='green', label ='Cluster 3')

# For K=4

In [ ]:
plt.scatter(data[y_kmeans==0, 0], data[y_kmeans==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(data[y_kmeans==1, 0], data[y_kmeans==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(data[y_kmeans==2, 0], data[y_kmeans==2, 1], s=100, c='green', label ='Cluster 3')
plt.scatter(data[y_kmeans==3, 0], data[y_kmeans==3, 1], s=100, c='cyan', label ='Cluster 4')

# For K=5

In [ ]:
plt.scatter(data[y_kmeans==0, 0], data[y_kmeans==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(data[y_kmeans==1, 0], data[y_kmeans==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(data[y_kmeans==2, 0], data[y_kmeans==2, 1], s=100, c='green', label ='Cluster 3')
plt.scatter(data[y_kmeans==3, 0], data[y_kmeans==3, 1], s=100, c='cyan', label ='Cluster 4')
plt.scatter(data[y_kmeans==4, 0], data[y_kmeans==4, 1], s=100, c='magenta', label ='Cluster 5')

# b) Draw a bar graph with X-axis as K value and Y-axis as silhouette score

In [ ]:
range_of_clusters=[2,3,4,5,6,7,8,9,10]
arr_sil=[]
for i_count in range_of_clusters:
    km=KMeans(n_clusters=i_count)
    km.fit(X3)
    cluster_labels= km.fit_predict(X_transformed)
    silhouette_avg = silhouette_score(X3, cluster_labels)
    print("For N_clusters = ",i_count,"The average silhouette_score is :",silhouette_avg)
    arr_sil.append(silhouette_avg)
plt.plot(range_of_clusters,arr_sil, 'bx-')
plt.bar(range_of_clusters,arr_sil)
plt.xlabel('Values of K') 
plt.ylabel('Silhouette score') 
plt.title('Silhouette analysis For Optimal k')
plt.show()

# 2 c)  Explain (1-2 lines) which is the best clustering and why.

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

fig, ax = plt.subplots(2, 2, figsize=(15,8))
for i in [2, 3, 4, 5]:
    '''
    Create KMeans instance for different number of clusters
    '''
    km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=42)
    q, mod = divmod(i, 2)
    '''
    Create SilhouetteVisualizer instance with KMeans instance
    Fit the visualizer
    '''
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(data)

# We can say that the clusters are well apart from each other as the silhouette score is closer to 1
# From above graph and score we can say that the optimal number of clusters is 3 as its silhouette score is greater than that of other 3 clusters.
# Silhouette Score(n=3 or n=3): 0.018937457053013588


# 3. Consider the per-processed title text of all articles and find words which frequently occur together. [2 points]

# We will find ngrams and Collocations

In [ ]:
from textblob import TextBlob
def combination_of_words(df):
    return (TextBlob(df['Title'][0]).ngrams(2))

In [ ]:
combination_of_words(df)

# Above 2 words are frequently occur together

In [ ]:
from nltk import bigrams
bigram_measures = nltk.collocations.BigramAssocMeasures()
df['Body-Collocation'] = df.apply(lambda df: BigramCollocationFinder.from_words(df['Title']),axis=1)
df['Body-Collocation'] = df['Body-Collocation'].apply(lambda df: df.nbest(bigram_measures.pmi, 3))
print (df['Body-Collocation'])

In [ ]:
from nltk import bigrams
bigram_measures = nltk.collocations.TrigramAssocMeasures()
df['Body-Collocation'] = df.apply(lambda df: TrigramCollocationFinder.from_words(df['Title']),axis=1)
df['Body-Collocation'] = df['Body-Collocation'].apply(lambda df: df.nbest(bigram_measures.pmi, 3))
print (df['Body-Collocation'])

# 3 a. find association rules of minimum

In [ ]:
data = list(df["Clean_Data"].apply(lambda x:x.split(",") ))
data

Apriori Algorithm and One-Hot Encoding

Apriori's algorithm transforms True/False or 1/0.
Using TransactionEncoder, we convert the list to a One-Hot Encoded Boolean list.
Products that customers bought or did not buy during shopping will now be represented by values 1 and 0

In [ ]:
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
df = pd.DataFrame(a_data,columns=a.columns_)
df = df.replace(False,0)
df

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
 
frequent_items = apriori(df, min_support=0.1, use_colnames=True)

frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_items, metric="lift", min_threshold=1)
rules.head()

import datetime
from datetime import datetime
start = datetime.now()
rules = association_rules(df, min_support = 0.001, max_length=2)
print('Execution time: ', datetime.now() - start)

In [ ]:
rules


In [ ]:
supps = rules["support"].values
sns.distplot(supps[supps > 0], kde=False, bins=np.arange(0, max(supps), 0.2))
plt.xlabel("Support")
plt.ylabel("Number of Rules")
plt.show()

In [ ]:
# Generate scatterplot using support and confidence
plt.figure(figsize=(10,6))
sns.scatterplot(x = "support", y = "confidence", 
                size = "lift", data = rules)
plt.margins(0.01,0.01)
plt.show()

In [ ]:
#Setting up the style
plt.figure(figsize = (15, 15))
plt.style.use('seaborn-white')
#Plotting the relationship between the metrics
plt.subplot(221)
sns.scatterplot(x="support", y="confidence",data=rules)
plt.subplot(222)
sns.scatterplot(x="support", y="lift",data=rules)
plt.subplot(223)
sns.scatterplot(x="confidence", y="lift",data=rules)
plt.subplot(224)
sns.scatterplot(x="antecedent support", y="consequent support",data=rules)

In [ ]:
# Function to convert rules to coordinates.
def rules_to_coordinates(rules):
    rules['antecedent'] = rules['antecedents'].apply(lambda antecedent: list(antecedent)[0])
    rules['consequent'] = rules['consequents'].apply(lambda consequent: list(consequent)[0])
    rules['rule'] = rules.index
    return rules[['antecedent','consequent','rule']]

In [ ]:
from pandas.plotting import parallel_coordinates

# Compute the frequent itemsets
frequent_itemsets = apriori(df, min_support = 0.20, 
                            use_colnames = True, max_len = 2)

# Compute rules from the frequent itemsets
rules = association_rules(frequent_itemsets, metric = 'confidence', 
                          min_threshold = 0.55)

# Convert rules into coordinates suitable for use in a parallel coordinates plot
coords = rules_to_coordinates(rules)

# Generate parallel coordinates plot
plt.figure(figsize=(4,8))
parallel_coordinates(coords, 'rule')
plt.legend([])
plt.grid(True)
plt.show()

In [ ]:
print(len(rules))

#  association rules  10 % support

df = apriori(df, min_support = 0.1, use_colnames = False, verbose = 1)
df

#  association rules  20% support

In [ ]:
#set a threshold value for the support value and calculate the support value.
df = apriori(df, min_support = 0.2, use_colnames = True, verbose = 1)
df

#  association rules  20% confidence

In [ ]:
#Let's view our interpretation values using the Associan rule function.
df_ar = association_rules(df, metric = "confidence", min_threshold = 0.2)
df_ar

In [ ]:
#Setting up the style
plt.figure(figsize = (15, 15))
plt.style.use('seaborn-white')
#Plotting the relationship between the metrics
plt.subplot(221)
sns.scatterplot(x="support", y="confidence",data=rules)
plt.subplot(222)
sns.scatterplot(x="support", y="lift",data=rules)
plt.subplot(223)
sns.scatterplot(x="confidence", y="lift",data=rules)
plt.subplot(224)
sns.scatterplot(x="antecedent support", y="consequent support",data=rules)